In [80]:
import bw2data as bd
import bw2io as bi
import pandas as pd

In [81]:
bd.projects.set_current("bw25_plca_grid_expansion")

In [82]:
bd.databases

Databases dictionary with 27 objects, including:
	ecoinvent-3.10-biosphere
	ecoinvent-3.10-cutoff
	ei310_SSP2_RCP19_2023
	ei310_SSP2_RCP19_2025
	ei310_SSP2_RCP19_2030
	ei310_SSP2_RCP19_2035
	ei310_SSP2_RCP19_2037
	ei310_SSP2_RCP19_2040
	ei310_SSP2_RCP19_2045
	ei310_SSP2_RCP26_2023
Use `list(this object)` to get the complete list.

Cleaning up:

In [83]:
new_databases = [
    "grid_components",
    "grid_components_2020",
    "grid_components_2023",
    "grid_expansion_2023",
    "grid_expansion_2025",
    "grid_expansion_2030",
    "grid_expansion_2035",
    "grid_expansion_2037",
    "grid_expansion_2040",
    "static_grid_expansion",
    "grid_expansion_prospective_2023",
    "grid_expansion_prospective_2025",
    "grid_expansion_prospective_2030",
    "grid_expansion_prospective_2035",
    "grid_expansion_prospective_2037",
    "grid_expansion_prospective_2040",
    "grid_expansion_static",
    "grid_expansion_prospective",
    "grid_components_2025",
    "grid_components_2030",
    "grid_components_2035",
    "grid_components_2037",
    "grid_components_2040",
    "grid_status_quo",
]
for db in new_databases:
    try:
        del bd.databases[db]
    except KeyError:
        pass

In [84]:
list(bd.databases)

['ecoinvent-3.10-biosphere',
 'ecoinvent-3.10-cutoff',
 'ei310_SSP2_RCP26_2023',
 'ei310_SSP2_RCP26_2025',
 'ei310_SSP2_RCP26_2030',
 'ei310_SSP2_RCP26_2035',
 'ei310_SSP2_RCP26_2037',
 'ei310_SSP2_RCP26_2040',
 'new_mixes',
 'ei310_SSP2_RCP26_2045',
 'grid_components_2045',
 'ei310_SSP2_RCP19_2023',
 'ei310_SSP2_RCP19_2025',
 'ei310_SSP2_RCP19_2030',
 'ei310_SSP2_RCP19_2035',
 'ei310_SSP2_RCP19_2037',
 'ei310_SSP2_RCP19_2040',
 'ei310_SSP2_RCP19_2045']

## Creating background databases

### Getting ecoinvent

In [85]:
# bi.import_ecoinvent_release(
#     version="3.10",
#     system_model="cutoff",
#     username="XXXXXXXXXXXXXX",
#     password="XXXXXXXXXXXXXX",
# )

### Creating prospective databases

In [86]:
# from premise import *

# ndb = NewDatabase(
#     scenarios=[
#         # {"model":"image", "pathway":"SSP2-RCP26", "year":2023},
#         # {"model":"image", "pathway":"SSP2-RCP26", "year":2025},
#         # {"model":"image", "pathway":"SSP2-RCP26", "year":2030},
#         # {"model":"image", "pathway":"SSP2-RCP26", "year":2035},
#         # {"model":"image", "pathway":"SSP2-RCP26", "year":2037},
#         # {"model":"image", "pathway":"SSP2-RCP26", "year":2040},
#         # {"model":"image", "pathway":"SSP2-RCP26", "year":2045},
#         #
#         {"model":"image", "pathway":"SSP2-RCP19", "year":2023},
#         {"model":"image", "pathway":"SSP2-RCP19", "year":2025},
#         {"model":"image", "pathway":"SSP2-RCP19", "year":2030},
#         {"model":"image", "pathway":"SSP2-RCP19", "year":2035},
#         {"model":"image", "pathway":"SSP2-RCP19", "year":2037},
#         {"model":"image", "pathway":"SSP2-RCP19", "year":2040},
#         {"model":"image", "pathway":"SSP2-RCP19", "year":2045},
#     ],
#     source_db="ecoinvent-3.10-cutoff", # <-- name of the database in the BW2 project. Must be a string.
#     source_version="3.10", # <-- version of ecoinvent. Can be "3.8", "3.9" or "3.10". Must be a string.
#     key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',    # to be requested from the library maintainers if you want ot use default scenarios included in `premise`
#     biosphere_name='ecoinvent-3.10-biosphere', # <-- name of the biosphere database in the BW2 project. Must be a string.
# )

# ndb.update()

# db_names = [
#     # 'ei310_SSP2_RCP26_2023',
#     # 'ei310_SSP2_RCP26_2025',
#     # 'ei310_SSP2_RCP26_2030',
#     # 'ei310_SSP2_RCP26_2035',
#     # 'ei310_SSP2_RCP26_2037',
#     # 'ei310_SSP2_RCP26_2040',
#     # 'ei310_SSP2_RCP26_2045',
#     #
#     'ei310_SSP2_RCP19_2023',
#     'ei310_SSP2_RCP19_2025',
#     'ei310_SSP2_RCP19_2030',
#     'ei310_SSP2_RCP19_2035',
#     'ei310_SSP2_RCP19_2037',
#     'ei310_SSP2_RCP19_2040',
#     'ei310_SSP2_RCP19_2045',
# ]

# ndb.write_db_to_brightway(db_names)


## Creating the grid database

### Preparation

In [87]:
def load_inputs_from_background(database_name: str):
    db = bd.Database(database_name)
    return {
        "glass_wool_mat": db.get(name="market for glass wool mat", location="GLO"),
        "porcelain": db.get(name="market for ceramic tile", location="GLO"),
        "epoxy": db.get(name="market for epoxy resin, liquid", location="RER"),
        "brass": db.get(name="market for brass", location="RoW"),
        "paint": db.get(name="market for electrostatic paint", location="GLO"),
        "paper": db.get(name="market for paper, melamine impregnated", location="RER"),
        "rubber": db.get(name="market for synthetic rubber", location="GLO"),
        "sulfur_hexafluoride": db.get(
            name="market for sulfur hexafluoride, liquid", location="RER"
        ),
        "lead": db.get(name="market for lead", location="GLO"),
        "bronze": db.get(name="market for bronze", location="GLO"),
        "polypropylene": db.get(
            name="market for polypropylene, granulate", location="GLO"
        ),
        "steel_sheet": db.get(name="market for sheet rolling, steel", location="GLO"),
        "steel_hot_rolled": db.get(
            name="market for steel, low-alloyed, hot rolled", location="GLO"
        ),
        "steel_lowalloyed": db.get(
            name="market for steel, low-alloyed", location="GLO"
        ),
        "steel_unalloyed": db.get(name="market for steel, unalloyed", location="GLO"),
        "transformer_oil": db.get(name="market for lubricating oil", location="RER"),
        "aluminium_wrought_alloy": db.get(
            name="market for aluminium, wrought alloy", location="GLO"
        ),
        "aluminium_sheet": db.get(
            name="market for sheet rolling, aluminium", location="GLO"
        ),
        "aluminium_cast": db.get(
            name="market for aluminium, cast alloy", location="GLO"
        ),
        "copper": db.get(name="market for copper, cathode", location="GLO"),
        "copper_wire": db.get(name="wire drawing, copper", location="RER"),
        "glass_fibre": db.get(name="market for glass fibre", location="GLO"),
        "kraft_paper": db.get(name="market for kraft paper", location="RER"),
        "softwood": db.get(
            name="market for sawnwood, softwood, raw, dried (u=20%)", location="RER"
        ),
        "concrete": db.get(
            name="market for concrete, normal strength", location="RoW"
        ),  # density of 2200 kg/m3 assumed for conversion, see Itten et al.
        "zinc": db.get(name="market for zinc", location="GLO"),
        "waste_concrete": db.get(
            name="market for waste concrete", location="Europe without Switzerland"
        ),
        "waste_polyethylene": db.get(
            name="market for waste polyethylene", location="DE"
        ),
        "waste_oil": db.get(
            name="market for waste mineral oil", location="Europe without Switzerland"
        ),
        "excavation": db.get(
            name="market for excavation, hydraulic digger", location="GLO"
        ),
        "polyester_resin": db.get(
            name="market for polyester resin, unsaturated", location="RER"
        ),
        "steel_chromium_steel": db.get(
            name="market for steel, chromium steel 18/8", location="GLO"
        ),
        "polycarbonate": db.get(name="market for polycarbonate", location="RER"),
        "wood": db.get(name="market for fibreboard, hard", location="RER"),
        "cement_unspecified": db.get(
            name="market for cement, unspecified", location="Europe without Switzerland"
        ),
        "glass_tube_borosilicate": db.get(
            name="market for glass tube, borosilicate", location="GLO"
        ),
        "extrusion_plastic_pipes": db.get(
            name="market for extrusion, plastic pipes", location="GLO"
        ),
        "polyethylene": db.get(
            name="market for polyethylene, low density, granulate", location="GLO"
        ),
        "copper_wire_drawing": db.get(
            name="market for wire drawing, copper", location="GLO"
        ),
    }

def load_inputs_from_background_alu_from_europe(database_name: str):
    db = bd.Database(database_name)
    return {
        "glass_wool_mat": db.get(name="market for glass wool mat", location="GLO"),
        "porcelain": db.get(name="market for ceramic tile", location="GLO"),
        "epoxy": db.get(name="market for epoxy resin, liquid", location="RER"),
        "brass": db.get(name="market for brass", location="RoW"),
        "paint": db.get(name="market for electrostatic paint", location="GLO"),
        "paper": db.get(name="market for paper, melamine impregnated", location="RER"),
        "rubber": db.get(name="market for synthetic rubber", location="GLO"),
        "sulfur_hexafluoride": db.get(
            name="market for sulfur hexafluoride, liquid", location="RER"
        ),
        "lead": db.get(name="market for lead", location="GLO"),
        "bronze": db.get(name="market for bronze", location="GLO"),
        "polypropylene": db.get(
            name="market for polypropylene, granulate", location="GLO"
        ),
        "steel_sheet": db.get(name="market for sheet rolling, steel", location="GLO"),
        "steel_hot_rolled": db.get(
            name="market for steel, low-alloyed, hot rolled", location="GLO"
        ),
        "steel_lowalloyed": db.get(
            name="steel production, converter, low-alloyed", location="RER"
        ),
        "steel_unalloyed": db.get(name="steel production, converter, unalloyed", location="RER"),
        "transformer_oil": db.get(name="market for lubricating oil", location="RER"),
        "aluminium_wrought_alloy": db.get(
            name="aluminium production, primary, ingot", location="IAI Area, EU27 & EFTA" # THIS CHANGED
        ),
        "aluminium_sheet": db.get(
            name="market for sheet rolling, aluminium", location="GLO"
        ),
        "aluminium_cast": db.get(
            name="market for aluminium, cast alloy", location="GLO"
        ),
        "copper": db.get(name="market for copper, cathode", location="GLO"),
        "copper_wire": db.get(name="wire drawing, copper", location="RER"),
        "glass_fibre": db.get(name="market for glass fibre", location="GLO"),
        "kraft_paper": db.get(name="market for kraft paper", location="RER"),
        "softwood": db.get(
            name="market for sawnwood, softwood, raw, dried (u=20%)", location="RER"
        ),
        "concrete": db.get(
            name="market for concrete, normal strength", location="RoW"
        ),  # density of 2200 kg/m3 assumed for conversion, see Itten et al.
        "zinc": db.get(name="market for zinc", location="GLO"),
        "waste_concrete": db.get(
            name="market for waste concrete", location="Europe without Switzerland"
        ),
        "waste_polyethylene": db.get(
            name="market for waste polyethylene", location="DE"
        ),
        "waste_oil": db.get(
            name="market for waste mineral oil", location="Europe without Switzerland"
        ),
        "excavation": db.get(
            name="market for excavation, hydraulic digger", location="GLO"
        ),
        "polyester_resin": db.get(
            name="market for polyester resin, unsaturated", location="RER"
        ),
        "steel_chromium_steel": db.get(
            name="market for steel, chromium steel 18/8", location="GLO"
        ),
        "polycarbonate": db.get(name="market for polycarbonate", location="RER"),
        "wood": db.get(name="market for fibreboard, hard", location="RER"),
        "cement_unspecified": db.get(
            name="market for cement, unspecified", location="Europe without Switzerland"
        ),
        "glass_tube_borosilicate": db.get(
            name="market for glass tube, borosilicate", location="GLO"
        ),
        "extrusion_plastic_pipes": db.get(
            name="market for extrusion, plastic pipes", location="GLO"
        ),
        "polyethylene": db.get(
            name="market for polyethylene, low density, granulate", location="GLO"
        ),
        "copper_wire_drawing": db.get(
            name="market for wire drawing, copper", location="GLO"
        ),
    }

In [88]:
# (code, name, unit) of grid components
COMPONENT_INFO = [
    ("disconnector", "Disconnector", "unit"),
    (
        "land_cable_oil_cu_150kv",
        "Land cable, oil insulated, copper, 150kV",
        "kilometer",
    ),
    #  ('live_tank_circuit_breaker', 'Live tank circuit breaker', 'unit'),
    ("overhead_line_400kv", "Overhead line, 400kV", "kilometer"),
    ("overhead_line_150kv", "Overhead line, 150kV", "kilometer"),
    ("overhead_line_10kv", "Overhead line, 10kV", "kilometer"),
    ("overhead_line_04kv", "Overhead line, 0.4kV", "kilometer"),
    ("overhead_line_HVDC", "Overhead line HVDC", "kilometer"),
    ("land_cable_oil_cu_HVDC", "Land cable, oil insulated, copper, HVDC", "kilometer"),
    #  ('plug_and_switch_system', 'Plug and switch system', 'unit'),
    #  ('surge_arrester', 'Surge arrester', 'unit'),
    ("transformer_250mva", "Transformer, 250MVA", "unit"),
    ("transformer_40mva", "Transformer, 40MVA", "unit"),
    ("transformer_315kva", "Transformer, 315kVA", "unit"),
    (
        "land_cable_vpe_al_04kv",
        "Land cable, vpe insulated, aluminium, 0.4kV",
        "kilometer",
    ),
    (
        "land_cable_vpe_al_10kv",
        "Land cable, vpe insulated, aluminium, 10kV",
        "kilometer",
    ),
    (
        "land_cable_vpe_al_10kv",
        "Land cable, vpe insulated, aluminium, 10kV",
        "kilometer",
    ),
    ("land_cable_epr_cu_11kv", "Land cable, epr insulated, copper, 11kV", "kilometer"),
    (
        "land_cable_vpe_al_50kv",
        "Land cable, vpe insulated, aluminium, 50kV",
        "kilometer",
    ),
    ("land_cable_vpe_cu_1kv", "Land cable, vpe insulated, copper, 1kV", "kilometer"),
    ("gas_insulated_switchgear_420kv", "Gas insulated switchgear, 420kV", "unit"),
    ("substation_lv", "Substation, LV", "unit"),
    ("substation_mv", "Substation, MV", "unit"),
    # ('office_building', 'Office building for grid operation', 'unit'),
    #  ('uniswitch', 'Uniswitch', 'unit')
]

In [89]:

def create_component_nodes(base_database_name: str, component_database_name: str, reset: bool=True):
    if reset:
        if component_database_name in bd.databases:
            del bd.databases[component_database_name]
        db_components = bd.Database(component_database_name)
        db_components.register()
    else :
        db_components = bd.Database(component_database_name)
    
    INPUT_NODES = load_inputs_from_background(base_database_name)
    
    for code, name, unit in COMPONENT_INFO:
        try:
            component = db_components.new_node(
                code=code, name=name, unit=unit, **{"reference product": name}
            )
        except:
            db_components.get(code).delete()
            component = db_components.new_node(
                code=code, name=name, unit=unit, **{"reference product": name}
            )
            component.save()
        component.save()
        component.new_edge(
            name=component["name"], input=component, amount=1, unit=unit, type="production"
        ).save()

        inputs_df = pd.read_csv(f"data/{code}.csv", sep=";", index_col="input")
        for input_code, amount, unit in zip(inputs_df.index, inputs_df.amount, inputs_df.unit):
            component.new_edge(
                input=INPUT_NODES[input_code], amount=amount, type="technosphere"
            ).save()

        if code == "gas_insulated_switchgear_420kv":
            sf6_node = bd.get_node(database="ecoinvent-3.10-biosphere", name="Sulfur hexafluoride", categories=("air",))
            component.new_edge(
                input=sf6_node,
                amount=28.6, # leakage
                type="biosphere",
            ).save()

### Creating grid nodes

In [90]:
SHARE_OVERHEAD_LINES_EHV_AC = 1
SHARE_OVERHEAD_LINES_EHV_DC = 0.05 
SHARE_OVERHEAD_LINES_HV = 0.9474
SHARE_OVERHEAD_LINES_MV = 0.1847
SHARE_OVERHEAD_LINES_LV = 0.0652

SHARE_AL_CABLES_EHV = 1
SHARE_AL_CABLES_HV = 0.5
SHARE_AL_CABLES_MV = 0.75
SHARE_AL_CABLES_LV = 0.75

grid_compositions = {
    2023: {
        # EHV AC
        "overhead_line_400kv": SHARE_OVERHEAD_LINES_EHV_AC * 36400,
        # EHV DC
        "overhead_line_HVDC": SHARE_OVERHEAD_LINES_EHV_DC * 2172,
        "land_cable_oil_cu_HVDC": (1 - SHARE_OVERHEAD_LINES_EHV_DC) * 2172,   
        # HV
        "overhead_line_150kv": SHARE_OVERHEAD_LINES_HV * 95200,
        "land_cable_vpe_al_50kv": (1 - SHARE_OVERHEAD_LINES_HV) * SHARE_AL_CABLES_HV * 95200, 
        "land_cable_oil_cu_150kv": (1 - SHARE_OVERHEAD_LINES_HV) * (1 - SHARE_AL_CABLES_HV) * 95200, 
        "transformer_250mva": 709, # EHV -> HV
        # "substation_mv": 709, # same as mv assumed, therefore included below
        # MV
        "overhead_line_10kv": SHARE_OVERHEAD_LINES_MV * 530200,
        "land_cable_vpe_al_10kv": (1 - SHARE_OVERHEAD_LINES_MV) * SHARE_AL_CABLES_MV * 530200,
        "land_cable_epr_cu_11kv": (1 - SHARE_OVERHEAD_LINES_MV) * (1 - SHARE_AL_CABLES_MV) * 117593,
        "transformer_40mva": 7278, # HV -> MV
        "gas_insulated_switchgear_420kv": 7278,
        "substation_mv": 7278 + 709, 
        # LV
        "overhead_line_04kv": SHARE_OVERHEAD_LINES_LV * 1570100,
        "land_cable_vpe_al_04kv": (1 - SHARE_OVERHEAD_LINES_LV) * SHARE_AL_CABLES_LV * 1570100,
        "land_cable_vpe_cu_1kv": (1 - SHARE_OVERHEAD_LINES_LV) * (1 - SHARE_AL_CABLES_LV) * 1570100,
        "transformer_315kva": 577790, # MV -> LV
        "substation_lv": 577790, 
    },
    2037: {
        # EHV AC
        "overhead_line_400kv": SHARE_OVERHEAD_LINES_EHV_AC * 11823,
        # EHV DC
        "overhead_line_HVDC": SHARE_OVERHEAD_LINES_EHV_DC * 17492,
        "land_cable_oil_cu_HVDC": (1 - SHARE_OVERHEAD_LINES_EHV_DC) * 17492,
        # HV
        "overhead_line_150kv": SHARE_OVERHEAD_LINES_HV * 28307,
        "land_cable_vpe_al_50kv": (1 - SHARE_OVERHEAD_LINES_HV) * SHARE_AL_CABLES_HV * 28307, 
        "land_cable_oil_cu_150kv": (1 - SHARE_OVERHEAD_LINES_HV) * (1 - SHARE_AL_CABLES_HV) * 28307, 
        "transformer_250mva": 221, # EHV -> HV
        # "substation_mv": 709/1.58, # same as mv assumed, therefore included below
        # MV
        "overhead_line_10kv": SHARE_OVERHEAD_LINES_MV * 117593,
        "land_cable_vpe_al_10kv": (1 - SHARE_OVERHEAD_LINES_MV) * SHARE_AL_CABLES_MV * 117593,
        "land_cable_epr_cu_11kv": (1 - SHARE_OVERHEAD_LINES_MV) * (1 - SHARE_AL_CABLES_MV) * 117593,
        "transformer_40mva": 1890, # HV -> MV
        "gas_insulated_switchgear_420kv": 1890,
        "substation_mv": (1890+709)/1.58, 
        
        # LV
        "overhead_line_04kv": SHARE_OVERHEAD_LINES_LV * 375511,
        "land_cable_vpe_al_04kv": (1 - SHARE_OVERHEAD_LINES_LV) * SHARE_AL_CABLES_LV * 375511,
        "land_cable_vpe_cu_1kv": (1 - SHARE_OVERHEAD_LINES_LV) * (1 - SHARE_AL_CABLES_LV) * 375511,
        "transformer_315kva": 133273, # MV -> LV
        "substation_lv": 133273, 
    },    
    2045: {
        # EHV AC
        "overhead_line_400kv": SHARE_OVERHEAD_LINES_EHV_AC * 250,
        # EHV DC
        "overhead_line_HVDC": SHARE_OVERHEAD_LINES_EHV_DC * 4683,
        "land_cable_oil_cu_HVDC": (1 - SHARE_OVERHEAD_LINES_EHV_DC) * 4683,
        # HV
        "overhead_line_150kv": SHARE_OVERHEAD_LINES_HV * 15096,
        "land_cable_vpe_al_50kv": (1 - SHARE_OVERHEAD_LINES_HV) * SHARE_AL_CABLES_HV * 15096, 
        "land_cable_oil_cu_150kv": (1 - SHARE_OVERHEAD_LINES_HV) * (1 - SHARE_AL_CABLES_HV) * 15096, 
        "transformer_250mva": 59, # EHV -> HV
        # "substation_mv": 709, # same as mv assumed, therefore included below
        # MV
        "overhead_line_10kv": SHARE_OVERHEAD_LINES_MV * 64837,
        "land_cable_vpe_al_10kv": (1 - SHARE_OVERHEAD_LINES_MV) * SHARE_AL_CABLES_MV * 64837,
        "land_cable_epr_cu_11kv": (1 - SHARE_OVERHEAD_LINES_MV) * (1 - SHARE_AL_CABLES_MV) * 117593,
        "transformer_40mva": 1022, # HV -> MV
        "gas_insulated_switchgear_420kv": 1022,
        "substation_mv": (1022+709)/1.58, 
        # LV
        "overhead_line_04kv": SHARE_OVERHEAD_LINES_LV * 198933,
        "land_cable_vpe_al_04kv": (1 - SHARE_OVERHEAD_LINES_LV) * SHARE_AL_CABLES_LV * 198933,
        "land_cable_vpe_cu_1kv": (1 - SHARE_OVERHEAD_LINES_LV) * (1 - SHARE_AL_CABLES_LV) * 198933,
        "transformer_315kva": 71943, # MV -> LV
        "substation_lv": 71943, 
    },
}

### Grid status quo

In [91]:
db_status_quo = bd.Database("grid_status_quo")
background_2023 = bd.Database("ei310_SSP2_RCP19_2023")

create_component_nodes(background_2023.name, db_status_quo.name)

node_code = 'grid_status_quo'
node_name = node_code

try:
    grid = db_status_quo.new_node(code=node_code, name=node_name)
except:
    existing_node = db_status_quo.get(node_code)
    existing_node.delete()
    grid = db_status_quo.new_node(code=node_code, name=node_name)
grid.save()
grid.new_edge(input=grid, amount=1, type='production').save()
for key, value in grid_compositions[2023].items():
    grid.new_edge(input=db_status_quo.get(key), amount=value, type='technosphere').save()

Aggregated material nodes for status quo, needed for the sankey diagram

In [92]:
material_exchanges = {
    'aluminium': {},
    'steel': {},
    'concrete': {},
    'copper': {},
    'plastics': {}
}

grid = bd.get_node(code="grid_status_quo")
for component_exchange in grid.technosphere():
    for material_exchange in component_exchange.input.technosphere():
        name = material_exchange.input['name'].lower()
        key = None
        if "aluminium" in name:
            key = 'aluminium'
        elif "steel" in name or "iron" in name:
            key = 'steel'
        elif "concrete" in name:
            key = 'concrete'
        elif "copper" in name:
            key = 'copper'
        elif "polyethylene" in name or "polypropylene" in name or "plastic" in name:
            key = 'plastics'

        total_amount = material_exchange.amount * component_exchange.amount
        
        if key:
            if material_exchange.input in material_exchanges[key]:
                material_exchanges[key][material_exchange.input] += total_amount
            else:
                material_exchanges[key][material_exchange.input] = total_amount
        else:
            if material_exchange.input in material_exchanges.setdefault('other', {}):
                material_exchanges['other'][material_exchange.input] += total_amount
            else:
                material_exchanges['other'][material_exchange.input] = total_amount


In [93]:
aggregated_material_nodes = []
for name, subexchanges in material_exchanges.items():
    try:
        mat = db_status_quo.new_node(
            name=f"aggregated material: {name}"
        )
    except:
        db_status_quo.get(name).delete()
        mat = db_status_quo.new_node(
            name=f"aggregated material: {name}"
        )
    mat.save()
    aggregated_material_nodes.append(mat)
    mat.new_exchange(input=mat, amount=1, type="production").save()
    for node, value in subexchanges.items():
        mat.new_exchange(input=node, amount=value, type="technosphere").save()

### Grid expansion nodes

Distributing grid expansion periods to sub-expansion periods that can be linked to prospective databases

In [94]:
# Current year
current_year = 2023
expansion_period_1 = 2037 - current_year
expansion_period_2 = 2045 - 2037

# Target years for the distribution
years_2037 = [2023, 2025, 2030, 2035, 2037]
timespan_2037 = years_2037[-1] - years_2037[0]
years_2045 = [2037, 2040, 2045]
timespan_2045 = years_2045[-1] - years_2045[0]

distributed_components = {}
for index, year in enumerate(years_2037):
    if year < 2037:
        year_data = {}
        duration = years_2037[index+1] - year 
        factor = duration / timespan_2037
        
        year_data = {}
        for component, total_value in grid_compositions[2037].items():
            year_data[component] = total_value * factor
        distributed_components[year] = year_data
        
for index, year in enumerate(years_2045):
    if year < 2045:
        year_data = {}
        duration = years_2045[index+1] - year 
        factor = duration / timespan_2045
        
        year_data = {}
        for component, total_value in grid_compositions[2045].items():
            year_data[component] = total_value * factor
        distributed_components[year] = year_data

In [95]:
import json

In [96]:
json.dump(distributed_components, open("data/distributed_components.json", "w"))

In [97]:
def test_distributed_sums(components, distributed_data):
    original_totals = {}
    for year in components:
        if year != 2023:
            for component, value in components[year].items():
                original_totals[component] = original_totals.get(component, 0) + value

    distributed_sums = {component: 0 for component in original_totals.keys()}
    for year in distributed_data.values():
        for component, value in year.items():
            distributed_sums[component] += value

    all_passed = True
    for component, total in original_totals.items():
        if not round(distributed_sums[component], 1) == round(total, 1):
            print(f"Test failed for {component}: distributed sum {distributed_sums[component]} != original total {total}")
            all_passed = False
    assert all_passed

test_result = test_distributed_sums(grid_compositions, distributed_components)

In [98]:
background_db_time_mapping = {
    'ei310_SSP2_RCP19_2023': 2023,
    'ei310_SSP2_RCP19_2025': 2025,
    'ei310_SSP2_RCP19_2030': 2030,
    'ei310_SSP2_RCP19_2035': 2035,
    'ei310_SSP2_RCP19_2037': 2037,
    'ei310_SSP2_RCP19_2040': 2040,
}

ends_of_expansion_periods = {
    2023: 2025,
    2025: 2030,
    2030: 2035,
    2035: 2037,
    2037: 2040,
    2040: 2045,
}

In [101]:
nodes_prospective_expansion = []
db_expansion = bd.Database("grid_expansion_prospective")
db_expansion.register()

for db_name, year in background_db_time_mapping.items():
    db_components = bd.Database(f"grid_components_{db_name.split("_")[2]}_{year}")
    db_background = bd.Database(db_name)

    create_component_nodes(db_background.name, db_components.name)

    node_code = f'grid_{ends_of_expansion_periods[year]}_prospective'
    node_name = node_code

    try:
        grid = db_expansion.new_node(code=node_code, name=node_name)
    except:
        existing_node = db_expansion.get(node_code)
        existing_node.delete()
        grid = db_expansion.new_node(code=node_code, name=node_name)
    grid.save()
    
    nodes_prospective_expansion.append(grid)
    
    grid.new_edge(input=grid, amount=1, type='production').save()
    for key, value in distributed_components[year].items():
        grid.new_edge(input=db_components.get(key), amount=value, type='technosphere').save()

SyntaxError: f-string: unmatched '(' (4270179220.py, line 6)

Static expansion for comparison:

In [76]:
nodes_static_expansion = []

db_static_expansion = bd.Database(f"grid_expansion_static")
db_static_expansion.register()

for year in background_db_time_mapping.values():

    node_code = f'grid_{ends_of_expansion_periods[year]}_static'
    node_name = node_code

    try:
        grid = db_static_expansion.new_node(code=node_code, name=node_name)
    except:
        existing_node = db_static_expansion.get(node_code)
        existing_node.delete()
        grid = db_static_expansion.new_node(code=node_code, name=node_name)
    grid.save()
    
    nodes_static_expansion.append(grid)
    
    grid.new_edge(input=grid, amount=1, type='production').save()
    for key, value in distributed_components[year].items():
        grid.new_edge(input=db_status_quo.get(key), amount=value, type='technosphere').save()

### Add electricity mixes for comparison of 1kWh

In [77]:
db_new_mixes = bd.Database("new_mixes")
db_new_mixes.register()

db = bd.Database("ei3.9.1-SSP2-RCP19-2045")

old_mix_2023 = bd.get_node(database="ei310_SSP2_RCP19_2023", name="market for electricity, low voltage", location="DE") # market for electricity, low voltage DE 2020
activity_code = 'el_2023_DE'
activity_name = 'market for electricity, high voltage DE 2023'
try:
    new_mix_2023 = db_new_mixes.new_node(code=activity_code, name=activity_name)
except:
    existing_activity = db_new_mixes.get(activity_code)
    existing_activity.delete()
    new_mix_2023 = db_new_mixes.new_activity(code=activity_code, name=activity_name, unit='unit', **{'reference product': activity_name})
new_mix_2023.save()

for exc in old_mix_2023.exchanges():
    if 'transmission network' in exc['name'] or 'distribution network' in exc['name'] or 'sulfur hexafluoride' in exc['name']:

        continue
    if exc['type'] == 'production':
        new_mix_2023.new_edge(input=new_mix_2023.key, amount=1, type='production').save()
    else:
        new_mix_2023.new_exchange(input=exc['input'], amount=exc['amount'], type=exc['type']).save()
        
        
old_mix_2045 = bd.get_node(database="ei310_SSP2_RCP26_2045", name="market for electricity, low voltage", location="DE") # market for electricity, low voltage DE 2020
activity_code = 'el_2045_DE'
activity_name = 'market for electricity, high voltage DE 2045'
try:
    new_mix_2045 = db_new_mixes.new_node(code=activity_code, name=activity_name, unit='unit', **{'reference product': activity_name})
except:
    existing_activity = db_new_mixes.get(activity_code)
    existing_activity.delete()
    new_mix_2045 = db_new_mixes.new_node(code=activity_code, name=activity_name, unit='unit', **{'reference product': activity_name})
new_mix_2045.save()

for exc in old_mix_2045.exchanges():
    if 'transmission network' in exc['name'] or 'distribution network' in exc['name'] or 'sulfur hexafluoride' in exc['name']:
        continue
    if exc['type'] == 'production':
        new_mix_2045.new_exchange(input=new_mix_2045.key, amount=1, type='production').save()
    else:
        new_mix_2045.new_exchange(input=exc['input'], amount=exc['amount'], type=exc['type']).save()
